# Análise Exploratória de Dados (EDA) - Temperatura da Superfície do Mar (SST)

Este notebook realiza a análise exploratória inicial do dataset NOAA OI SST V2 High Resolution.

**Dataset:** `sst.mon.mean.nc`
**Fonte:** NOAA PSL THREDDS Server

**Nota:** Devido ao tamanho do dataset (~2GB), algumas operações (como o cálculo da média global) podem consumir muita memória. Esta versão utiliza `chunks` para carregar os dados de forma mais eficiente e calcula a média global sobre um subconjunto temporal para evitar problemas de memória.

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import os

# Definir diretório base do projeto
project_dir = "/home/ubuntu/plankton_project"
report_dir = os.path.join(project_dir, "report")
data_dir = os.path.join(project_dir, "data")

# Criar diretório de report se não existir
os.makedirs(report_dir, exist_ok=True)

print(f"Diretório de report: {report_dir}")
print("Bibliotecas importadas com sucesso!")

## 1. Carregar o Dataset com Chunking

In [ ]:
# Caminho para o ficheiro NetCDF
sst_file_path = os.path.join(data_dir, "sst.mon.mean.nc")

# Carregar o dataset usando xarray com chunks para otimizar memória
# Ajustar o tamanho dos chunks conforme necessário
try:
    ds_sst = xr.open_dataset(sst_file_path, chunks={"time": 12}) # Chunks de 12 meses (1 ano)
    print(f"Dataset SST carregado com sucesso de {sst_file_path} usando chunks.")
except FileNotFoundError:
    print(f"Erro: Ficheiro não encontrado em {sst_file_path}")
except Exception as e:
    print(f"Erro ao carregar o dataset: {e}")

## 2. Inspeção Inicial do Dataset

In [ ]:
# Exibir a estrutura do dataset
if 'ds_sst' in locals() and ds_sst is not None:
    print("Estrutura do Dataset SST:")
    print(ds_sst)
else:
    print("Dataset não foi carregado corretamente.")

In [ ]:
# Verificar as variáveis disponíveis
if 'ds_sst' in locals() and ds_sst is not None:
    print("\nVariáveis:")
    for var in ds_sst.data_vars:
        print(f"- {var}: {ds_sst[var].attrs.get('long_name', 'N/A')}")

    print("\nCoordenadas:")
    for coord in ds_sst.coords:
        print(f"- {coord}")

    print("\nAtributos Globais:")
    print(ds_sst.attrs)

## 3. Análise Preliminar

In [ ]:
if 'ds_sst' in locals() and ds_sst is not None:
    # Cobertura Temporal
    time_min = ds_sst['time'].min().values
    time_max = ds_sst['time'].max().values
    print(f"Cobertura Temporal: {np.datetime_as_string(time_min, unit='D')} a {np.datetime_as_string(time_max, unit='D')}")

    # Cobertura Espacial
    lat_min, lat_max = ds_sst['lat'].min().values, ds_sst['lat'].max().values
    lon_min, lon_max = ds_sst['lon'].min().values, ds_sst['lon'].max().values
    print(f"Cobertura Espacial (Lat): {lat_min}° a {lat_max}°")
    print(f"Cobertura Espacial (Lon): {lon_min}° a {lon_max}°")

    # Verificar unidades da variável SST
    sst_units = ds_sst['sst'].attrs.get('units', 'N/A')
    print(f"Unidades de SST: {sst_units}")

## 4. Visualização Inicial (com Otimização de Memória)

In [ ]:
if 'ds_sst' in locals() and ds_sst is not None:
    # Selecionar um subconjunto temporal (ex: últimos 5 anos) para calcular a média global
    try:
        latest_time = ds_sst['time'].max()
        start_date_subset = latest_time - np.timedelta64(5*365, 'D') # Aproximadamente 5 anos atrás
        sst_subset = ds_sst['sst'].sel(time=slice(start_date_subset, latest_time))

        print("Calculando a média temporal para os últimos 5 anos...")
        sst_mean_subset = sst_subset.mean(dim='time').compute()
        print("Cálculo da média concluído.")

        # Plotar o mapa da SST média (subconjunto)
        plt.figure(figsize=(12, 6))
        sst_mean_subset.plot(cmap='viridis')
        plt.title(f'Temperatura Média Global da Superfície do Mar (SST) - Média Mensal (Últimos 5 anos)')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.tight_layout()
        # Salvar a figura com caminho absoluto
        save_path_map = os.path.join(report_dir, 'sst_mean_subset_map.png')
        plt.savefig(save_path_map)
        print(f"Mapa da SST média (subconjunto) salvo em {save_path_map}")
        plt.close() # Fechar a figura para libertar memória

    except Exception as e:
        print(f"Erro ao calcular ou plotar a média do subconjunto: {e}")

In [ ]:
if 'ds_sst' in locals() and ds_sst is not None:
    # Selecionar um ponto para a série temporal (ex: próximo ao Equador no Atlântico)
    lat_point = 0
    lon_point = 330 # Equivalente a -30 Oeste

    try:
        print(f"Extraindo série temporal para o ponto ({lat_point}°, {lon_point-360}°W)...")
        sst_timeseries = ds_sst['sst'].sel(lat=lat_point, lon=lon_point, method='nearest').compute()
        print("Extração da série temporal concluída.")

        # Plotar a série temporal
        plt.figure(figsize=(12, 4))
        sst_timeseries.plot()
        plt.title(f'Série Temporal da SST Mensal no Ponto ({lat_point}°, {lon_point-360}°W)')
        plt.xlabel('Data')
        plt.ylabel(f'SST ({sst_units})')
        plt.grid(True)
        plt.tight_layout()
        # Salvar a figura com caminho absoluto
        save_path_ts = os.path.join(report_dir, 'sst_timeseries_point.png')
        plt.savefig(save_path_ts)
        print(f"Série temporal da SST salva em {save_path_ts}")
        plt.close() # Fechar a figura para libertar memória

    except Exception as e:
        print(f"Erro ao extrair ou plotar a série temporal: {e}")

## 5. Próximos Passos

- Análise mais detalhada das distribuições.
- Tratamento de valores ausentes.
- Cálculo de anomalias.
- Exploração de padrões sazonais e tendências.
- Download e EDA do dataset de Clorofila-a.
- Processamento e EDA do dataset de Plâncton.